In [1]:
# Your code and text here
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
# More code follows...


## Example markdown text
This section can contain explanations and code comments.
This is how you format markdown in Jupyter notebooks.


In [2]:
# Further code follows...
def example_function(x):
    return x**2
example_function(4)